# Aufgabe:
Sage, basierend auf den gegebenen Daten, vorher, ob das Einkommen einer Person über 50 Tausend Dollar pro Jahr liegt.  
Datensatz von [kaggle.com](https://www.kaggle.com/datasets/uciml/adult-census-income)

In [1]:
# Import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix

In [2]:
# Laden des Datensatzes 
df = pd.read_csv('datasets/adult.csv')

df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [3]:
df.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Dieser erste kurze Blick bringt einige wichtige Punkte zum Vorschein.  
Worum genau handelt es sich bei capital loss, respektive capital gain und fnlwgt?  
Fehlende Werte, scheinbar gekennzeichnet durch '?' bei kategorischen Spalten, müssen behandelt werden.  
Wie wurden fehlende Werte bei numerischen Merkmalen vermerkt? Eventuell mit '0'? Gibt es keine?  
Doppeltes Merkmal zur Bildung in education und education-num.

In [5]:
# Umbenennen der Attribute
df.rename(columns={
    'fnlwgt': 'final_weight',
    'education.num': 'education_num',
    'marital.status': 'marital_status',
    'capital.gain': 'capital_gain',
    'capital.loss': 'capital_loss',
    'hours.per.week': 'hours_per_week',
    'native.country': 'native_country'
}, inplace=True)

df.columns

Index(['age', 'workclass', 'final_weight', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [6]:
df.replace(to_replace='?', value=None, inplace=True)

In [7]:
# Droppen derer Datenpunkte bei denen Attributwerte fehlen.
df.dropna(axis=0, inplace=True)

In [8]:
# Festlegen der Features die wir für unsere Vorhersage nutzen wollen.
features = [
    'age',
    'workclass',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country'
]

X = df[features]
y = df.income.replace(to_replace={'>50K': 1, '<=50K': 0})

Wir haben an dieser Stelle drei Entscheidungen getroffen: 
1. Wir beachten die Datenpunkte bei denen workclass oder occupation unbekannt ist zunächst nicht. Man könnte die dort fehlenden Werte durch Imputation ergänzen, was meiner Meinung nach allerdings keinen Sinn ergeben würde und unseren Datensatz verfälschen würde.
2. Wir verwenden die Attribute final_weight und education nicht. Education, weil wir weiterhin education_num nutzen und somit einer Wiederholung des Attributs vorbeugen. Final_weight nicht, weil nicht genau bekannt ist was dieses Attribut widerspiegelt und wir somit die Chance auf Target Leakage reduzieren.
3. Wir verwenden die beiden Attribute capital_loss und capital_gain als Merkmale für unser Modell.

In [9]:
categorical_cols = [cname for cname in X.columns if 
                    X[cname].dtype == 'object']

numerical_cols = [cname for cname in X.columns if
                  X[cname].dtype == 'int64']

In [10]:
# Teilen des Datensatzes in Train- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8, test_size=0.2,
                                                    random_state=42)

Wir teilen unsere Daten in Trainings- und Testdaten um anschließend auf den Trainingsdaten Kreuzvalidation durchführen zu können.  
So bestimmen wir die besten Hyperparameter für unser Modell um es dann mit allen Trainingsdaten fitten zu können und schlussendlich die Performance auf neuen Daten (Testdaten) zu bewerten.

In [11]:
# Funktion zum bewerten eines einzelnen Modells
def score_model(max_iter):
    # Preprocessing Schritte..
    # .. für numerische Daten
    numerical_transformer = MinMaxScaler()
    
    # .. für kategorische Daten
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
    # Preprocessing zusammengefasst
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
    ])
    
    # Definiere das Modell
    model = LogisticRegression(max_iter=max_iter, random_state=42)
    
    # Erstelle die Pipeline
    current_pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                                     ('model', model)
                                    ])
    
    # Bewerte das Modell über Kreuzvalidation und anhand mehrere Metriken
    scores = cross_val_score(current_pipeline, X_train, y_train,
                             cv=5, n_jobs=8,
                             scoring='f1')
    return scores.mean()

In [12]:
# Finde beste Hyperparameter für die Logistische Regression
results = {100 * i: score_model(100 * i) for i in range(1, 9)}

best_max_iter = max(results, key=results.get)

print(f"Die beste logistische Regression bekommen wir bei einer maximalen Anzahl an Iterationen von {best_max_iter}")

results

Die beste logistische Regression bekommen wir bei einer maximalen Anzahl an Iterationen von 100


{50: 0.662771605750242,
 100: 0.6628747507747567,
 150: 0.662469883444365,
 200: 0.6626502416552126,
 250: 0.6626502416552126,
 300: 0.6626502416552126,
 350: 0.6626502416552126,
 400: 0.6626502416552126}

In [16]:
# Trainiere das neue Modell auf dem vollständigen Trainingsdatensatz
numerical_transformer = MinMaxScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
    ])

model = LogisticRegression(max_iter=best_max_iter)

final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

final_pipeline.fit(X_train, y_train)

predictions = final_pipeline.predict(X_test)

conf_matrix = confusion_matrix(y_test, predictions)

conf_matrix

D:\JupyterProjects\jupytervenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[4185,  348],
       [ 616,  884]], dtype=int64)

In [18]:
# Berechne Precision, Recall und Accuracy auf dem Testdatensatz
precision = 884 / (884 + 616)
recall = 884 / (884 + 348)
accuracy = (4185 + 884) / (4185 + 348 + 616 + 884)

precision, recall, accuracy

(0.5893333333333334, 0.7175324675324676, 0.8402121664180342)